<a href="https://colab.research.google.com/github/aditya161989/EIP4.0/blob/master/Final_Normal_PersonAttrubutes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16


from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras import regularizers
from keras import optimizers
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers.convolutional import Convolution2D, MaxPooling2D, SeparableConv2D
from keras.layers import Activation, Flatten, Dense, Dropout, SpatialDropout2D, UpSampling2D, GlobalAveragePooling2D, AveragePooling2D

from keras.models import load_model

import numpy as np
import os


Using TensorFlow backend.


In [0]:
# Initialize model parameters

# Training parameters
epochs = 20
data_augmentation = True

# Weight loading required
load_saved_weights = True

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Epoch count
epoch_count = 20

# Filepath for loading saved weights
filepath="/content/gdrive/My Drive/Model/newaugweights-05-7.88.hdf5"

# Filepath for saving weights
savefilepath="/content/gdrive/My Drive/Model/newaugweights-{epoch:02d}-{val_loss:.2f}.hdf5"

weight_decay = 4e-3

In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.shape

(13573, 9)

In [0]:
# Since the model is being trained multiple times using best saved weights,
# the train test split is saved in drive so that the model never trains on test data
test_indices = list(np.load('/content/gdrive/My Drive/test_indices.npy'))


In [0]:
len(test_indices)

2036

In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
0,resized/1.jpg,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
1,resized/2.jpg,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0
2,resized/3.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,resized/4.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4,resized/5.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0


In [0]:
# Cut out implementation

# p : the probability that random erasing is performed
# s_l, s_h : minimum / maximum proportion of erased area against input image
# r_1, r_2 : minimum / maximum aspect ratio of erased area
# v_l, v_h : minimum / maximum value for erased area
# pixel_level : pixel-level randomization for erased area

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.1, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :] = c

        return input_img

    return eraser

In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        # print(image.shape)
        if data_augmentation == True:
          eraser = get_random_eraser(v_h = 1)
          img_gen = ImageDataGenerator()
          for idx in range(image.shape[0]):
            image[idx] = image[idx].astype('float32') / 255
            image[idx] = eraser(image[idx])
            image[idx] = img_gen.apply_transform(image[idx], transform_parameters = {'tx': .1, 'ty': .1})
        if index == (self.__len__() - 1):
          self.on_epoch_end()
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [0]:
val_df = one_hot_df.iloc[test_indices]

In [0]:

train_indices = [i for i in range(0,one_hot_df.shape[0]) if i not in test_indices]

In [0]:
train_df = one_hot_df.iloc[train_indices]

In [0]:
train_df.shape

(11680, 28)

In [0]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
0,resized/1.jpg,0,1,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
2,resized/3.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0
3,resized/4.jpg,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0
4,resized/5.jpg,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
5,resized/6.jpg,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=64)
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
# Transform the image to 56X56 ASAP
inputlayer = Input(shape=(224,224,3))
start = Conv2D(32, (3, 3),  kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay), input_shape=(224,224,3), padding='same')(inputlayer)
start1 = Conv2D(64, (3, 3),  kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay), padding='same')(start)
pool = AveragePooling2D()(start1)
mid = Conv2D(32, (3, 3),  kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay), padding='same')(pool)
mid1 = Conv2D(64, (3, 3),  kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay), padding='same')(mid)
pool2 = AveragePooling2D()(mid1)
end = Conv2D(8,1,1)(pool2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (1, 1))`
  


In [0]:

def newBody(num_classes=10, name='demo'):
  layer1 = SeparableConv2D(32, (3,3), padding='same', kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay), input_shape=(56, 56, 8))(end)
  layer1 = Activation('relu')(layer1)
  layer1 = BatchNormalization()(layer1)

  layer2 = SeparableConv2D(64, (3,3), padding='same', kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay))(layer1)
  layer2 = Activation('relu')(layer2)
  layer2 = BatchNormalization()(layer2)

  layer2 = AveragePooling2D()(layer2)#28
  layer2 = Dropout(0.1)(layer2)
  layer2 = SeparableConv2D(32,1,1,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay))(layer2)

  layer3 = SeparableConv2D(32, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer2)
  layer3 = Activation('relu')(layer3)
  layer3 = BatchNormalization()(layer3)

  layer4 = SeparableConv2D(64, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer3)
  layer4 = Activation('relu')(layer4)
  layer4 = BatchNormalization()(layer4)

  layer4 = AveragePooling2D()(layer4)#14
  layer4 = Dropout(0.1)(layer4)
  layer4 = SeparableConv2D(32,1,1,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay))(layer4)

  layer5 = SeparableConv2D(32, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer4)
  layer5 = Activation('relu')(layer5)
  layer5 = BatchNormalization()(layer5)

  layer6 = SeparableConv2D(64, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer5)
  layer6 = Activation('relu')(layer6)
  layer6 = BatchNormalization()(layer6)

  layer6 = AveragePooling2D()(layer6)#7
  layer6 = Dropout(0.1)(layer6)
  layer6 = SeparableConv2D(32,1,1,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay))(layer6)

  layer7 = SeparableConv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay))(layer6)
  layer7 = Activation('relu')(layer7)
  layer7 = BatchNormalization()(layer7)

  layer8 = SeparableConv2D(64, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer7)
  layer8 = Activation('relu')(layer8)
  layer8 = BatchNormalization()(layer8)

  layer8 = AveragePooling2D()(layer8)#7
  layer8 = Dropout(0.1)(layer8)
  layer8 = SeparableConv2D(32,1,1,kernel_initializer='he_normal',kernel_regularizer=regularizers.l2(weight_decay))(layer8)

  
  layer9 = SeparableConv2D(32, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer8)
  layer9 = Activation('relu')(layer9)
  layer9 = BatchNormalization()(layer9)

  layer10 = SeparableConv2D(64, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer9)
  layer10 = Activation('relu')(layer10)
  layer10 = BatchNormalization()(layer10)

  layer11 = AveragePooling2D()(layer10)#3
  layer11 = Dropout(0.1)(layer11)
  layer11 = SeparableConv2D(32,1,1,kernel_regularizer=regularizers.l2(weight_decay))(layer11)  

  layer12 = SeparableConv2D(64, (3,3), padding='same',kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(weight_decay))(layer11)

  layer13 = Flatten()(layer12)
  outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal', name=f"{name}_output")(layer13)
  return outputs

In [0]:

def build_head(name):
    return newBody(num_classes=num_units[name], name= name)
    

# heads
gender = build_head("gender")
image_quality = build_head("image_quality")
age = build_head("age")
weight = build_head("weight")
bag = build_head("bag")
footwear = build_head("footwear")
emotion = build_head("emotion")
pose = build_head("pose")

# print(image_quality)

model = Model(
    inputs=inputlayer, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

print(model.count_params())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (1, 1), kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (1, 1), kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (1, 1), kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:49: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(32, (1, 1), kernel_initializer="he_normal", kernel_regularizer=<keras.reg...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:62: U

320835


In [0]:
def scheduler(epoch, lr): #Implements cyclical learning rate
  step_size = 4300 #One triangle is completed at the 46th epoch and the learning rate remains the base rate for the rest 4 epochs
  iterations = epoch * 182
  base_lr = 0.00025 #base learning rate
  max_lr = .001 #max learning rate
  cycle = np.floor(1+iterations/(2*step_size))
  x = np.abs(iterations/step_size - 2*cycle + 1)
  new_lr = base_lr + (max_lr-base_lr)*np.maximum(0, (1-x))
  print((lr, new_lr))
  if(round(lr, 5) <= base_lr and epoch > 1):
    return base_lr 
  else:
    return new_lr

In [0]:
from keras.callbacks import Callback

class MyLogger(Callback):
    def on_epoch_end(self, epoch, logs=None):
        avg = ( logs['val_gender_output_acc'] + logs['val_image_quality_output_acc'] + logs['val_age_output_acc'] + logs['val_weight_output_acc'] + logs['val_bag_output_acc'] + logs['val_footwear_output_acc'] + logs['val_pose_output_acc'] + logs['val_emotion_output_acc'])*100/8
        print(avg) 

In [0]:
# checkpoint
checkpoint = ModelCheckpoint(savefilepath, monitor='val_loss', save_weights_only=True, verbose=1)
mylogger = MyLogger()

In [0]:

callbacks_list = [checkpoint, mylogger]

In [0]:
# def load_weights(model, filepath):
#   return model.load_weights(filepath)


In [0]:
# Loss Definition
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",
#   "bag_output": "binary_crossentropy",
# 	"footwear_output": "binary_crossentropy",
# 	"emotion_output": "binary_crossentropy",
# 	"pose_output": "binary_crossentropy",

# }
loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0, "weight_output": 1.0, "bag_output": 1.0, "footwear_output": 1.0,"emotion_output": 1.0, "pose_output": 1.0}

In [0]:
# Execute when weights have been saved

opt = SGD(lr=0.00025, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy", 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
if load_saved_weights == True:
  model.load_weights(filepath)

In [0]:
results = model.evaluate_generator(valid_gen, verbose =1)
dict(zip(model.metrics_names,results))

31/31 [==============================] - 25s 791ms/step


{'age_output_acc': 0.3870967741935484,
 'age_output_loss': 1.4171747546042166,
 'bag_output_acc': 0.5700604838709677,
 'bag_output_loss': 0.9117189261221117,
 'emotion_output_acc': 0.7142137096774194,
 'emotion_output_loss': 0.8971969427600983,
 'footwear_output_acc': 0.5292338709677419,
 'footwear_output_loss': 0.9682077342464078,
 'gender_output_acc': 0.5866935483870968,
 'gender_output_loss': 0.6707130670547485,
 'image_quality_output_acc': 0.5383064516129032,
 'image_quality_output_loss': 1.1432928423727713,
 'loss': 7.880903736237557,
 'pose_output_acc': 0.6421370967741935,
 'pose_output_loss': 0.8890646792227223,
 'weight_output_acc': 0.6436491935483871,
 'weight_output_loss': 0.9705355532707707}

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=20,
    verbose=1,
    callbacks = callbacks_list
)

Epoch 1/20
182/182 [==============================] - 203s 1s/step - loss: 7.6646 - gender_output_loss: 0.6478 - image_quality_output_loss: 0.9500 - age_output_loss: 1.4144 - weight_output_loss: 0.9752 - bag_output_loss: 0.9057 - footwear_output_loss: 0.9576 - pose_output_loss: 0.8991 - emotion_output_loss: 0.9017 - gender_output_acc: 0.6035 - image_quality_output_acc: 0.5541 - age_output_acc: 0.4020 - weight_output_acc: 0.6360 - bag_output_acc: 0.5692 - footwear_output_acc: 0.5450 - pose_output_acc: 0.6180 - emotion_output_acc: 0.7120 - val_loss: 7.7568 - val_gender_output_loss: 0.6739 - val_image_quality_output_loss: 1.0171 - val_age_output_loss: 1.4167 - val_weight_output_loss: 0.9643 - val_bag_output_loss: 0.9250 - val_footwear_output_loss: 0.9625 - val_pose_output_loss: 0.8893 - val_emotion_output_loss: 0.8952 - val_gender_output_acc: 0.5539 - val_image_quality_output_acc: 0.5373 - val_age_output_acc: 0.3866 - val_weight_output_acc: 0.6431 - val_bag_output_acc: 0.5741 - val_footwe

In [0]:
 model